In [1]:
!pip install duckdb
!pip install notebook
!pip install jupysql pandas matplotlib duckdb-engine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext sql

In [3]:
%sql duckdb:///data/local.duckdb


Connecting to 'duckdb:///data/local.duckdb'

In [4]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb:///data/local.duckdb'

Success


In [5]:
%%sql

CREATE TABLE point_data AS
SELECT *
FROM ST_Read("./data/geojson/cemetery.geojson")

Running query in 'duckdb:///data/local.duckdb'

Count


In [6]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT centroid: geom.ST_Centroid(),
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    locationX: centroid.ST_X(),
    locationy: centroid.ST_Y(),
    description: substr(md5(random()::VARCHAR), 1, 10)


Running query in 'duckdb:///data/local.duckdb'

Count


In [7]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb:///data/local.duckdb'

id,locationX,locationy,description
1,71.60207635350525,54.78643014239917,fe87c66f47
2,68.07547055464238,40.23959944486414,f17997da1c
3,22.71972581720911,48.451325419849496,d709fd0462
4,29.491294654319063,46.07657549119183,52278b8ef2
5,79.38921734504402,52.74566659600047,c349f5d23e
6,6.359217615158059,43.142364551113644,86b33e7b89
7,1.9459106666618595,49.15726517327247,39f3052f2e
8,7.758160430065118,43.808794503076086,35ade03ae0
9,-0.9758063970803335,51.474531240130844,d596222a4b
10,66.95163331926233,39.698323306344484,d886dfd4de


In [8]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb:///data/local.duckdb'

count_star()
402325


In [9]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb:///data/local.duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,402325,377825,201163.0,116141.36786333569,100582,199807,300008,402325,0.00
locationX,DOUBLE,-179.12087778314643,179.80886915,326174,1.0731648788767683,60.204031265910395,-45.97290389276277,9.428730099239099,28.661680675202255,402325,0.00
locationy,DOUBLE,-66.52681067468725,78.92123011264881,477667,38.35798860494107,20.333799419387596,36.726646213046266,44.57695979989656,49.94175374799608,402325,0.00
description,VARCHAR,00006c18b1,ffffebd8ba,492084,None,None,None,None,None,402325,0.00


In [11]:
%sql COPY point_data_centroid TO './data/point_data_centroid.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb:///data/local.duckdb'

Count
